In [4]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MultipleLocator
import pandas as pd
import commonfxns as cf, OAPBuoyData as OAP, mplRC, OAPBuoyComp as bc,viz, evalfxns as ev, \
    frequencyBands as fb
import netCDF4 as nc
import cftime
import datetime as dt
import cmocean
import gsw
import pickle
import os
import matplotlib as mpl
from IPython.display import Markdown, display, HTML
import astropy.timeseries as apt
from collections import OrderedDict
import warnings
from matplotlib.colors import ListedColormap
import xarray as xr
import glob
mplRC.paperRC2()

In [2]:
dir_hindcast = '/space/hall5/sitestore/eccc/crd/ccrn/model_output/CMIP6/final/CMIP6/DCPP/CCCma/CanESM5/dcppA-hindcast' ### dir to dcppA_hindcast

In [5]:
realization  = []    ### extract available realizations
for dir in glob.glob(dir_hindcast + '/*'):
    name = dir.split('/')[-1]
    realization.append(name.split('-')[-1])
realization = np.unique(realization)
print(f'Available realizations : \n {realization}')

Available realizations : 
 ['r10i1p2f1' 'r11i1p2f1' 'r12i1p2f1' 'r13i1p2f1' 'r14i1p2f1' 'r15i1p2f1'
 'r16i1p2f1' 'r17i1p2f1' 'r18i1p2f1' 'r19i1p2f1' 'r1i1p2f1' 'r20i1p2f1'
 'r21i1p2f1' 'r22i1p2f1' 'r23i1p2f1' 'r24i1p2f1' 'r25i1p2f1' 'r26i1p2f1'
 'r27i1p2f1' 'r28i1p2f1' 'r29i1p2f1' 'r2i1p2f1' 'r30i1p2f1' 'r31i1p2f1'
 'r32i1p2f1' 'r33i1p2f1' 'r34i1p2f1' 'r35i1p2f1' 'r36i1p2f1' 'r37i1p2f1'
 'r38i1p2f1' 'r39i1p2f1' 'r3i1p2f1' 'r40i1p2f1' 'r4i1p2f1' 'r5i1p2f1'
 'r6i1p2f1' 'r7i1p2f1' 'r8i1p2f1' 'r9i1p2f1']


In [7]:
year=1998
dsets = {}
for r in realization:
    try:
        path = glob.glob(dir_hindcast + '/' + f's{year}-' + r + '/Omon/chlos/gn/v20190429/*nc')
        dsets[r] = xr.open_dataset(path[0])
    except:
        pass

In [12]:
len(dsets.keys()),len(realization)

(20, 40)

In [13]:
hindcast_d = xr.concat([ds for key, ds in dsets.items()], dim = 'member').assign_coords(member = list(dsets.keys()) )

In [14]:
hindcast_d

<xarray.Dataset> Size: 1GB
Dimensions:             (member: 20, time: 120, bnds: 2, j: 291, i: 360,
                         vertices: 4)
Coordinates:
  * time                (time) object 960B 1999-01-16 12:00:00 ... 2008-12-16...
  * j                   (j) int32 1kB 0 1 2 3 4 5 6 ... 285 286 287 288 289 290
  * i                   (i) int32 1kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
    latitude            (j, i) float64 838kB -78.39 -78.39 ... 50.23 50.01
    longitude           (j, i) float64 838kB 73.5 74.5 75.5 ... 72.96 72.99
  * member              (member) <U9 720B 'r10i1p2f1' 'r11i1p2f1' ... 'r9i1p2f1'
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (member, time, bnds) object 38kB 1999-01-01 00:00:00 ...
    vertices_latitude   (member, j, i, vertices) float64 67MB -78.29 ... 50.11
    vertices_longitude  (member, j, i, vertices) float64 67MB 74.0 74.0 ... 73.0
    chlos               (member, time, j, i) float32 1GB nan nan nan ... nan nan
Attributes: (12/53)
    CCCma_model_hash:            Unknown
    CCCma_parent_runid:          d2a-asm-e10
    CCCma_pycmor_hash:           13db8596c37129e414cad7ae31f2927ca8f5dd39
    CCCma_runid:                 d2a199901e10
    Conventions:                 CF-1.7 CMIP-6.2
    YMDH_branch_time_in_child:   1999:01:01:00
    ...                          ...
    variable_id:                 chlos
    variant_label:               r10i1p2f1
    version:                     v20190429
    license:                     CMIP6 model data produced by The Government ...
    cmor_version:                3.4.0
    tracking_id:                 hdl:21.14100/63947217-36b2-4b9b-84d0-535a528...

In [ ]:
####### load data based on intialization year #####
hindcast_dict = {}
for year in tqdm(range(1998,2020)):
    dsets = {}
    for r in realization:
        try:
            path = glob.glob(dir_hindcast + '/' + f's{year}-' + r + '/Omon/chlos/gn/v20190429/*nc')
            dsets[r] = xr.open_dataset(path[0])
        except:
            pass
    hindcast_dict[year] = xr.concat([ds for key, ds in dsets.items()], dim = 'member').assign_coords(member = list(dsets.keys()) )
###### regrid data based on initialization year #####
ds_out = xe.util.grid_global(1, 1)  ### definde regrider
for key, ds in tqdm(hindcast_dict.items()):
    regridder = xe.Regridder(ds, ds_out, 'bilinear', ignore_degenerate=True, periodic=True)
    ds = coords_edit(regridder(ds.chlos))
    ds = ds.rename({'time': 'lead_time', 'member' : 'ensembles'})
    ds['lead_time'] = np.arange(1,121)
    ds = ds.assign_coords(year = key+1).transpose('ensembles','lead_time','lat','lon')
    ds.to_dataset(name = 'chlos').to_netcdf(out_dir + f'chlos_Omon_{key}_ensmebles_{key+1}01_{key+10}12_1x1_LE.nc')